# Sub-agents Guide

Learn how to leverage specialized AI assistants (sub-agents) in Claude Code for different tasks.

## What Are Sub-agents?

Sub-agents are specialized AI assistants that Claude can delegate tasks to. Each sub-agent:
- Has a **dedicated purpose** (e.g., code review, testing, documentation)
- Operates in an **isolated context** (separate conversation thread)
- Has **configurable tool access** (read-only, write, execute, etc.)
- Follows **custom instructions** tailored to its role
- Can be invoked **automatically** or **explicitly**

Think of sub-agents as specialized team members with specific expertise!

## Built-in Sub-agents

Claude Code comes with several built-in sub-agents:

### 1. Plan Subagent
**Purpose:** Research and analysis without making changes

**When it activates:**
- When you enter Plan Mode (Shift+Tab twice)
- Exploration and codebase understanding tasks

**Capabilities:**
- Search files and analyze code structure
- Gather context for planning
- Read-only operations
- Prevents infinite agent nesting

**Example use:**
```
[Shift+Tab twice]
"How is authentication implemented in this project?"
```

### 2. Explore Subagent
**Purpose:** Fast, token-efficient codebase exploration

**When it activates:**
- Powered by Haiku (faster, cheaper model)
- When searching for patterns or files
- Research-oriented tasks

**Capabilities:**
- Quick file discovery
- Pattern searching
- Context-efficient exploration

**Example use:**
```
"Find all files that handle database connections"
```

### 3. General-Purpose Agent
**Purpose:** Main conversation thread

**Responsibilities:**
- Primary interface with you
- Delegates to specialized sub-agents
- Coordinates overall workflow
- Makes final decisions

## Creating Custom Sub-agents

You can create specialized sub-agents for your specific needs!

### File Structure

Sub-agents are defined in markdown files:

```
Project-level:  .claude/agents/agent-name.md
User-level:     ~/.claude/agents/agent-name.md
Plugin-provided: (via installed plugins)
```

### Basic Template

```markdown
---
name: code-reviewer
description: Reviews code for security vulnerabilities and best practices
allowed-tools: [Read, Grep, Glob]
model: inherit
---

You are an expert code reviewer specializing in security and best practices.

When reviewing code, focus on:
1. Security vulnerabilities (SQL injection, XSS, CSRF, etc.)
2. Code quality and maintainability
3. Best practices for the language/framework
4. Performance issues
5. Error handling

Provide specific line numbers and actionable feedback.
```

### Frontmatter Fields

| Field | Required | Description |
|-------|----------|-------------|
| `name` | Yes | Unique identifier for the sub-agent |
| `description` | Yes | When to invoke this agent (be specific!) |
| `allowed-tools` | No | Tools this agent can use |
| `model` | No | AI model (sonnet, opus, haiku, inherit) |

## Example Custom Sub-agents

### Example 1: Security Auditor

```markdown
---
name: security-auditor
description: Audits code for security vulnerabilities and compliance issues
allowed-tools: [Read, Grep, Glob]
model: sonnet
---

You are a security expert specializing in application security.

Audit code for:
- OWASP Top 10 vulnerabilities
- Authentication/authorization flaws
- Data validation issues
- Sensitive data exposure
- Cryptography misuse
- Dependency vulnerabilities

For each issue found:
1. Severity level (Critical/High/Medium/Low)
2. Exact location (file:line)
3. Explanation of the risk
4. Remediation steps
```

**Usage:**
```
"Audit the authentication module for security issues"
```

### Example 2: Test Generator

```markdown
---
name: test-generator
description: Generates comprehensive unit and integration tests
allowed-tools: [Read, Write, Edit, Grep, Bash(pytest:*)]
model: sonnet
---

You are a test automation expert.

Generate tests that:
- Cover happy path scenarios
- Cover edge cases and error conditions
- Use appropriate testing frameworks
- Follow AAA pattern (Arrange, Act, Assert)
- Include helpful comments
- Aim for 90%+ code coverage

For each function tested:
- Test valid inputs
- Test invalid inputs
- Test boundary conditions
- Test error handling
```

**Usage:**
```
"Generate comprehensive tests for the UserService class"
```

### Example 3: Documentation Writer

```markdown
---
name: doc-writer
description: Creates comprehensive documentation for code and APIs
allowed-tools: [Read, Write, Grep, Glob]
model: sonnet
---

You are a technical documentation expert.

Create documentation that includes:
- Clear, concise descriptions
- Usage examples with code
- Parameter descriptions
- Return value documentation
- Error handling information
- Common use cases

Follow these standards:
- Use markdown formatting
- Include code examples
- Write for the target audience
- Be technically accurate
```

**Usage:**
```
"Create API documentation for all routes in api/v1/"
```

### Example 4: Performance Optimizer

```markdown
---
name: perf-optimizer
description: Analyzes and optimizes code for better performance
allowed-tools: [Read, Grep, Glob, Bash(time:*), Bash(profile:*)]
model: sonnet
---

You are a performance optimization expert.

Analyze code for:
- Time complexity issues
- Space complexity issues
- Unnecessary operations
- N+1 query problems
- Inefficient algorithms
- Memory leaks

Provide:
1. Current performance characteristics
2. Bottlenecks identified
3. Optimization suggestions
4. Expected improvement
5. Trade-offs to consider
```

**Usage:**
```
"Analyze the data processing pipeline for performance issues"
```

## Tool Permissions

### Understanding Tool Access

Control what each sub-agent can do by specifying allowed tools.

### Read-Only Agents
```markdown
allowed-tools: [Read, Grep, Glob, LS]
```
**Use for:** Reviewers, auditors, analyzers

### Write Access Agents
```markdown
allowed-tools: [Read, Write, Edit, Grep, Glob]
```
**Use for:** Implementers, generators, refactorers

### Execution Access Agents
```markdown
allowed-tools: [Read, Write, Edit, Bash(npm test:*), Bash(pytest:*)]
```
**Use for:** Test runners, build managers

### Full Access Agents
```markdown
allowed-tools: ["*"]
```
**Use for:** General-purpose agents (use sparingly!)

### Specific Command Patterns
```markdown
allowed-tools: [
  Read,
  Write,
  "Bash(git status:*)",
  "Bash(git diff:*)",
  "Bash(npm test:*)"
]
```
**Use for:** Limiting bash commands to specific patterns

## Model Selection

### Available Models

| Model | Speed | Cost | Best For |
|-------|-------|------|----------|
| **Opus** | Slowest | Highest | Complex reasoning, critical decisions |
| **Sonnet** | Medium | Medium | Most tasks, balanced performance |
| **Haiku** | Fastest | Lowest | Quick searches, simple tasks |
| **inherit** | - | - | Use parent session's model |

### Choosing the Right Model

**Use Opus for:**
- Complex architectural decisions
- Security audits
- Critical code reviews

**Use Sonnet for:**
- General code review
- Feature implementation
- Documentation generation
- Most sub-agents (default choice)

**Use Haiku for:**
- File searching
- Pattern matching
- Quick exploration
- Cost-sensitive operations

**Use inherit for:**
- Consistency with main session
- Dynamic model selection

## Invocation Strategies

### Automatic Invocation

Claude automatically invokes sub-agents based on the `description` field.

**Good descriptions:**
```markdown
description: Reviews code for security vulnerabilities, OWASP issues, and best practices when user requests security audit or code review
```

**Bad descriptions:**
```markdown
description: Reviews code  # Too vague!
```

**Tips for automatic invocation:**
- Include trigger keywords users might say
- Be specific about the agent's expertise
- Mention both what it does AND when to use it

### Explicit Invocation

You can explicitly request a specific sub-agent:

```
"Use the security-auditor agent to review auth.py"
"Have the test-generator create tests for UserService"
"Ask the doc-writer to document the API endpoints"
```

## Real-World Sub-agent Examples

### Language-Specific Agents

#### Python Expert
```markdown
---
name: python-expert
description: Python code review and best practices for Pythonic code, PEP 8, type hints
allowed-tools: [Read, Write, Edit, Grep, Bash(python:*), Bash(pytest:*)]
model: sonnet
---

You are a Python expert focused on Pythonic code.

Review for:
- PEP 8 compliance
- Type hints usage
- List comprehensions where appropriate
- Context managers
- Exception handling best practices
- Python 3.9+ features
```

#### JavaScript/TypeScript Expert
```markdown
---
name: js-expert
description: JavaScript/TypeScript code review, modern ES6+ features, TypeScript best practices
allowed-tools: [Read, Write, Edit, Grep, Bash(npm:*)]
model: sonnet
---

You are a JavaScript/TypeScript expert.

Review for:
- Modern ES6+ syntax
- Async/await patterns
- TypeScript type safety
- Error handling
- Performance patterns
```

### Framework-Specific Agents

#### React Expert
```markdown
---
name: react-expert
description: React component review, hooks usage, performance optimization for React applications
allowed-tools: [Read, Write, Edit, Grep]
model: sonnet
---

You are a React expert.

Focus on:
- Hooks best practices
- Component composition
- Performance optimization (memo, useMemo, useCallback)
- Accessibility
- Testing with React Testing Library
```

### DevOps Agents

#### Docker Expert
```markdown
---
name: docker-expert
description: Docker and containerization best practices, Dockerfile optimization, docker-compose
allowed-tools: [Read, Write, Edit, Bash(docker:*)]
model: sonnet
---

You are a Docker and containerization expert.

Focus on:
- Multi-stage builds
- Layer caching
- Security best practices
- Image size optimization
- docker-compose configuration
```

## Best Practices

### 1. Clear, Specific Descriptions
```markdown
✅ description: Reviews Python code for PEP 8 compliance, type hints, and Pythonic patterns when user requests code review or Python optimization
❌ description: Reviews Python code
```

### 2. Principle of Least Privilege
```markdown
✅ Reviewers get read-only: [Read, Grep, Glob]
❌ Everyone gets full access: ["*"]
```

### 3. Use Appropriate Models
```markdown
✅ Explorers use haiku for speed
✅ Reviewers use sonnet for quality
✅ Critical security audits use opus
```

### 4. Focused Instructions
```markdown
✅ Specific checklist of what to review
❌ "Review the code and make it better"
```

### 5. Reusable Across Projects
```
✅ Store general-purpose agents in ~/.claude/agents/
✅ Store project-specific agents in .claude/agents/
```

### 6. Test Before Deploying
```
1. Create the agent
2. Test with sample code
3. Verify tool permissions work
4. Check automatic invocation
5. Deploy to team
```

## Practical Exercises

### Exercise 1: Create a Code Reviewer (15 min)

1. Create `.claude/agents/my-reviewer.md`
2. Define it to review code for:
   - Code quality
   - Best practices
   - Potential bugs
3. Give it read-only permissions
4. Test it on a sample file

### Exercise 2: Create a Test Generator (20 min)

1. Create a test-generator agent
2. Give it read and write permissions
3. Configure it to use pytest (or your testing framework)
4. Test it by generating tests for a simple function

### Exercise 3: Language-Specific Expert (15 min)

1. Create an expert for your primary language
2. Focus it on framework-specific patterns
3. Include common pitfalls to watch for
4. Test with real code from your project

### Exercise 4: Automatic vs Explicit Invocation (10 min)

1. Create an agent with a good description
2. Ask Claude to perform a task that should trigger it
3. Verify the agent was invoked automatically
4. Try invoking it explicitly by name

## Troubleshooting

### Agent Not Being Invoked
**Problem:** Claude doesn't use your custom agent

**Solutions:**
1. Check description includes trigger keywords
2. Make description more specific
3. Try explicit invocation by name
4. Verify the file is in correct location

### Invalid YAML Syntax
**Problem:** Agent file doesn't load

**Solutions:**
1. Validate YAML frontmatter
2. Check for typos in field names
3. Ensure proper formatting
4. Review error messages

### Tool Permission Errors
**Problem:** Agent can't perform expected actions

**Solutions:**
1. Check `allowed-tools` list
2. Verify tool names are correct
3. Test with broader permissions first
4. Review tool usage patterns

### Wrong Model Selected
**Problem:** Agent is too slow or fast

**Solutions:**
1. Adjust `model` field
2. Use haiku for speed
3. Use sonnet for balance
4. Use opus for quality

## Key Takeaways

1. **Specialized Agents:** Create focused sub-agents for specific tasks
2. **Tool Permissions:** Use principle of least privilege
3. **Good Descriptions:** Include trigger keywords for automatic invocation
4. **Model Selection:** Match model to task complexity
5. **Reusability:** Share agents across projects
6. **Testing:** Verify agents work before deploying

## Next Steps

Now that you understand sub-agents, continue to:
- **Notebook 03:** Hooks - Automate workflows with event triggers
- **Notebook 04:** Skills - Create reusable capability packages
- **Notebook 05:** Slash Commands - Build custom commands
